<a href="https://colab.research.google.com/github/vanessaaleung/rawdata/blob/master/scrape_crunchbase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/vanessaaleung/rawdata/master/updated_apr_10.csv")

In [ ]:
df["Industry"] = df["Headquarters Location"] = df["Headquarters Regions"] = df["Founded Date"] = df["Founders"] = df["Funding Status"] = \
                df["Last Funding Type"] = df["Number of Employees"] = df["Revenue Range"] = \
                df["Hug Tags"] = df["Ipo Status"] = df["Company Type"] = df["Number of Acquisitions"] = \
                df["Total Funding"] = df["Number of Investments"] = None
df.head()

,Company name,Notes,Status,Last modified,Where to apply,Industry,Location,Founded Date,Founders,Funding Status,Last Funding Type,Number of Employees,Revenue Range,Hug Tags,Ipo Status,Company Type,Number of Acquisitions,Total Funding,Headquarters Location,Headquarters Regions
0,GBKSOFT,\n,Hiring freeze,2020-04-10T16:27:59.000Z,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1A- Product Ventures,"Product, dev, design, agile coach, user resear...",Hiring,2020-04-07T16:04:33.000Z,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,1Password,1Password is still hiring and growing. They cu...,Hiring,2020-04-07T19:18:31.000Z,https://1password.com/jobs,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,1stdibs,-1 report of hiring freeze\n4 reports of layoffs,Hiring freeze,2020-04-09T21:28:21.000Z,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,1sts,\n,Hiring freeze,2020-04-06T16:29:48.000Z,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
driver = webdriver.Chrome(executable_path="/Users/eric/Documents/chromedriver")
driver.get("https://www.crunchbase.com/login")

In [ ]:
username = "ypqi@ucdavis.edu"
password = "66893220c"
email = driver.find_element_by_xpath("//*[@id='mat-input-1']").send_keys(username)
time.sleep(5)
pwd = driver.find_element_by_xpath("//*[@id='mat-input-2']").send_keys(password)
time.sleep(5)
submit = driver.find_element_by_xpath("//*[@id='mat-tab-content-0-0']/div/login/form/div/button[2]").click()
time.sleep(5)

In [ ]:
count = 0
for index, row in df.iloc[454:500, :].iterrows():
    count += 1
    if count == 30:
        time.sleep(30)
        count = 0
    company_name = row["Company name"].strip()
    search = driver.find_element_by_xpath("//*[@id='mat-input-0']")
    search.clear()
    search.send_keys(company_name)
    search.send_keys(Keys.RETURN)
    time.sleep(10)
    try:
        print("Scraping the {}th company".format(index + 1))
        first_result = driver.find_element_by_xpath("/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/search/page-layout/div/div/form/div[2]/results/div/div/div[3]/sheet-grid/div/div/grid-body/div/div/grid-row[1]")
        while first_result.find_element_by_tag_name("a").get_attribute('title').lower() != company_name.strip().lower():
            first_result = first_result.find_element_by_xpath("following-sibling::*")
        url = first_result.find_element_by_tag_name("a").get_attribute('href')
        driver.get(url)

        content = driver.find_element_by_class_name("section-layout-content")
        first_field_card = content.find_element_by_xpath("//*[@id='section-overview']/mat-card/div[2]/fields-card[1]")
        cats = first_field_card.find_elements_by_css_selector("span.cb-text-color-medium")
        for cat in cats:
            if cat.find_element_by_tag_name("span").text.strip() == "Industries":
                df.at[index,'Industry'] = cat.find_element_by_xpath("following-sibling::*").text
            if cat.find_element_by_tag_name("span").text.strip() == "Headquarters Location":
                df.at[index,'Headquarters Location'] = cat.find_element_by_xpath("following-sibling::*").text
            if cat.find_element_by_tag_name("span").text.strip() == "Headquarters Regions":
                df.at[index,'Headquarters Regions'] = cat.find_element_by_xpath("following-sibling::*").text
            if cat.find_element_by_tag_name("span").text.strip() == "Founded Date":
                df.at[index,'Founded Date'] = cat.find_element_by_xpath("following-sibling::*").text
            if cat.find_element_by_tag_name("span").text.strip() == "Founders":
                df.at[index,'Founders'] = cat.find_element_by_xpath("following-sibling::*").text
            if cat.find_element_by_tag_name("span").text.strip() == "Funding Status":
                df.at[index,'Funding Status'] = cat.find_element_by_xpath("following-sibling::*").text
            if cat.find_element_by_tag_name("span").text.strip() == "Last Funding Type":
                df.at[index,'Last Funding Type'] = cat.find_element_by_xpath("following-sibling::*").text
            if cat.find_element_by_tag_name("span").text.strip() == "Number of Employees":
                df.at[index,'Number of Employees'] = cat.find_element_by_xpath("following-sibling::*").text
            if cat.find_element_by_tag_name("span").text.strip() == "Estimated Revenue Range":
                df.at[index,'Estimated Revenue Range'] = cat.find_element_by_xpath("following-sibling::*").text
            if cat.find_element_by_tag_name("span").text.strip() == "Hub Tags":
                df.at[index,'Hub Tags'] = cat.find_element_by_xpath("following-sibling::*").text

        second_field_card = content.find_element_by_xpath("//*[@id='section-overview']/mat-card/div[2]/fields-card[2]")
        cats = second_field_card.find_elements_by_css_selector("span.cb-text-color-medium")
        for cat in cats:
            if cat.find_element_by_tag_name("span").text.strip() == "IPO Status":
                df.at[index,'IPO Status'] = cat.find_element_by_xpath("following-sibling::*").text
            if cat.find_element_by_tag_name("span").text.strip() == "Company Type":
                df.at[index,'Company Type'] = cat.find_element_by_xpath("following-sibling::*").text

        big_values_card = content.find_element_by_xpath("//*[@id='section-overview']/mat-card/div[2]/big-values-card")
        divs = big_values_card.find_elements_by_tag_name("div")
        for div in divs:
            if div.find_element_by_tag_name("span").text.strip() == "Number of Acquisitions":
                df.at[index,'Number of Acquisitions'] = div.find_element_by_tag_name("span").find_element_by_xpath("following-sibling::*").text
            if div.find_element_by_tag_name("span").text.strip() == "Total Funding Amount":
                df.at[index,'Total Funding Amount'] = div.find_element_by_tag_name("span").find_element_by_xpath("following-sibling::*").text
            if div.find_element_by_tag_name("span").text.strip() == "Number of Investments":
                df.at[index,'Number of Investments'] = div.find_element_by_tag_name("span").find_element_by_xpath("following-sibling::*").text
        time.sleep(10)
    except Exception as e:
        print(company_name, "can't find info")
        not_found.append(company_name)
        

In [ ]:
df[df["Industry"].isna()][:50]

,Company name,Notes,Status,Last modified,Where to apply,Industry,Location,Founded Date,Founders,Funding Status,...,Company Type,Number of Acquisitions,Total Funding,Headquarters Location,Headquarters Regions,Number of Investments,Estimated Revenue Range,IPO Status,Total Funding Amount,Hub Tags
1,1A- Product Ventures,"Product, dev, design, agile coach, user resear...",Hiring,2020-04-07T16:04:33.000Z,NaN,None,None,None,None,None,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
4,1sts,\n,Hiring freeze,2020-04-06T16:29:48.000Z,NaN,None,None,None,None,None,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
6,2U,Multiple reports of a hiring freeze. Details b...,Hiring freeze,2020-04-06T18:15:57.000Z,NaN,None,None,None,None,None,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
8,3dhubs,\n,Hiring,2020-03-31T20:12:01.000Z,https://www.3dhubs.com/jobs/,None,None,None,None,None,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
11,3N,\n,Hiring,2020-04-01T01:39:14.000Z,NaN,None,None,None,None,None,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
14,3rd I visuals,Business development\n,Hiring,2020-04-06T19:59:09.000Z,NaN,None,None,None,None,None,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
15,4 Degrees,\n,Hiring,2020-04-06T17:29:41.000Z,NaN,None,None,None,None,None,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
19,7-11,\n,Hiring,2020-04-01T16:16:21.000Z,NaN,None,None,None,None,None,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
21,8x8,\n,"Hiring,Hiring freeze",2020-04-08T18:17:17.000Z,NaN,None,None,None,None,None,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
22,10x Genomics,Actively hiring across the organisation\n,"Hiring,Hiring freeze",2020-04-04T14:59:34.000Z,https://www.10xgenomics.com/careers/,None,None,None,None,None,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN


In [ ]:
df.to_csv("updated_apr_10.csv", index=False)

In [ ]:
df[df["Company name"] == "Adobe"]["Industry"] = 

In [ ]:
driver = webdriver.Chrome(executable_path="/Users/eric/Documents/chromedriver")
driver.get("https://www.crunchbase.com/organization/adobe-systems")

In [ ]:
try:
    first_result = driver.find_element_by_xpath("/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/search/page-layout/div/div/form/div[2]/results/div/div/div[3]/sheet-grid/div/div/grid-body/div/div/grid-row[1]")
    while first_result.find_element_by_tag_name("a").get_attribute('title').lower() != company_name.strip().lower():
        first_result = first_result.find_element_by_xpath("following-sibling::*")
    url = first_result.find_element_by_tag_name("a").get_attribute('href')
    driver.get(url)

    content = driver.find_element_by_class_name("section-layout-content")
    first_field_card = content.find_element_by_xpath("//*[@id='section-overview']/mat-card/div[2]/fields-card[1]")
    cats = first_field_card.find_elements_by_css_selector("span.cb-text-color-medium")
    for cat in cats:
        if cat.find_element_by_tag_name("span").text.strip() == "Industries":
            df[df["Company name"] == "Adobe"]["Industry"] = cat.find_element_by_xpath("following-sibling::*").text
        if cat.find_element_by_tag_name("span").text.strip() == "Headquarters Location":
            df[df["Company name"] == "Adobe"]["Headquarters Location"] = cat.find_element_by_xpath("following-sibling::*").text
        if cat.find_element_by_tag_name("span").text.strip() == "Headquarters Regions":
            df[df["Company name"] == "Adobe"]["Headquarters Regions"] = cat.find_element_by_xpath("following-sibling::*").text
        if cat.find_element_by_tag_name("span").text.strip() == "Founded Date":
            df[df["Company name"] == "Adobe"]["Founded Date"] = cat.find_element_by_xpath("following-sibling::*").text
        if cat.find_element_by_tag_name("span").text.strip() == "Founders":
            df[df["Company name"] == "Adobe"]["Founders"] = cat.find_element_by_xpath("following-sibling::*").text
        if cat.find_element_by_tag_name("span").text.strip() == "Funding Status":
            df[df["Company name"] == "Adobe"]["Funding Status"] = cat.find_element_by_xpath("following-sibling::*").text
        if cat.find_element_by_tag_name("span").text.strip() == "Last Funding Type":
            df[df["Company name"] == "Adobe"]["Last Funding Type"] = cat.find_element_by_xpath("following-sibling::*").text
        if cat.find_element_by_tag_name("span").text.strip() == "Number of Employees":
            df[df["Company name"] == "Adobe"]["Number of Employees"] = cat.find_element_by_xpath("following-sibling::*").text
        if cat.find_element_by_tag_name("span").text.strip() == "Estimated Revenue Range":
            df[df["Company name"] == "Adobe"]["Estimated Revenue Range"] = cat.find_element_by_xpath("following-sibling::*").text
        if cat.find_element_by_tag_name("span").text.strip() == "Hub Tags":
            df[df["Company name"] == "Adobe"]["Hub Tags"] = cat.find_element_by_xpath("following-sibling::*").text

    second_field_card = content.find_element_by_xpath("//*[@id='section-overview']/mat-card/div[2]/fields-card[2]")
    cats = second_field_card.find_elements_by_css_selector("span.cb-text-color-medium")
    for cat in cats:
        if cat.find_element_by_tag_name("span").text.strip() == "IPO Status":
            df[df["Company name"] == "Adobe"]["IPO Status"] = cat.find_element_by_xpath("following-sibling::*").text
        if cat.find_element_by_tag_name("span").text.strip() == "Company Type":
            df[df["Company name"] == "Adobe"]["Company Type"] = cat.find_element_by_xpath("following-sibling::*").text

    big_values_card = content.find_element_by_xpath("//*[@id='section-overview']/mat-card/div[2]/big-values-card")
    divs = big_values_card.find_elements_by_tag_name("div")
    for div in divs:
        if div.find_element_by_tag_name("span").text.strip() == "Number of Acquisitions":
            df[df["Company name"] == "Adobe"]["Number of Acquisitions"] = div.find_element_by_tag_name("span").find_element_by_xpath("following-sibling::*").text
        if div.find_element_by_tag_name("span").text.strip() == "Total Funding Amount":
            df[df["Company name"] == "Adobe"]["Total Funding Amount"] = div.find_element_by_tag_name("span").find_element_by_xpath("following-sibling::*").text
        if div.find_element_by_tag_name("span").text.strip() == "Number of Investments":
            df[df["Company name"] == "Adobe"]["Number of Investments"] = div.find_element_by_tag_name("span").find_element_by_xpath("following-sibling::*").text
    time.sleep(10)
except Exception as e:
        print("Adobe", "can't find info")

Adobe can't find info
